# Desafío Técnico - Mercado Libre

#### Ezequiel Panzarasa

# Análisis exploratorio de datos

## Armado de Dataset
Después de explorar un poco la API pública de Mercado Libre, se decidió optar por el armado de un dataset de 10.000 registros. Siendo que la API tiene un offset máximo de 1000, se decidió tomar las 10 categorías más populares (de acuerdo a la página principal de MercadoLibre. Ver imagen adjunta) y armar el dataset con 1.000 registros de cada categoría. <br>
Se eligieron estas 10 categorías siguiendo la hipótesis de que a mayor popularidad se tiene un mayor número de ventas, y más representativa resulta la información obtenida en la muestra para cada una de las publicaciones. 

#### Siendo que el armado del dataset mediante la API de Mercado Libre es un proceso costoso en términos de tiempo, se decidió separarlo del entregable final. Este proceso se ejecuta una sola vez, guardando los resultados en un CSV (raw_dataset.csv), y el entregable final levanta este CSV. 

In [3]:
# Imports
import pandas as pd
import numpy as np
import requests

In [4]:
# Diccionario de categorías
categories = {
    "computacion" : "MLA1648",
    "electrodomesticos" : "MLA5726",
    "deportes" : "MLA1276",
    "belleza" : "MLA1246",
    "juegos" : "MLA1132",
    "celulares" : "MLA1051",
    "electronica" : "MLA1000",
    "ropa" : "MLA1430",
    "hogar" : "MLA1574",
    "consolas" : "MLA1144"
}

In [5]:
# url base para hacer las requests
req_url = "https://api.mercadolibre.com/sites/MLA/search?category="
# Flag para inicializar el df
first_pass = True  

for cat_value in categories.values():  # Loop through 10 categories
    
    i = 0
    while i <= 1000:  # use i as offset counter.
        request_string = req_url + cat_value + "&offset={}".format(i)
        request = requests.get(request_string)
        results = request.json()["results"] # get results from JSON
        
        if first_pass:  # Si es la primer pasada, inicializo results_df.
            results_df = pd.json_normalize(results)
            first_pass = False  # toggle flag luego de la incialización. 
            
        else:  # Si no es la primer pasada, append to results_df.
            df_to_append = pd.json_normalize(results)
            results_df = results_df.append(df_to_append, ignore_index=True)
        
        i += 50

In [7]:
# Visualize df
results_df

,id,site_id,title,price,sale_price,currency_id,available_quantity,sold_quantity,buying_mode,listing_type_id,...,seller.seller_reputation.metrics.claims.excluded.real_rate,seller.seller_reputation.metrics.claims.excluded.real_value,seller.seller_reputation.metrics.cancellations.excluded.real_rate,seller.seller_reputation.metrics.cancellations.excluded.real_value,seller.seller_reputation.protection_end_date,seller.seller_reputation.real_level,seller.eshop.eshop_rubro.id,seller.eshop.eshop_rubro.name,seller.eshop.eshop_rubro.category_id,seller.car_dealer_logo
0,MLA900810750,MLA,Repetidor Tp-link Tl-wa850re Blanco 1 Unidad,2055.00,None,ARS,83,3179,buy_it_now,gold_special,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MLA856313404,MLA,Disco Sólido Interno Kingston Sa400s37/240g 240gb,3688.00,None,ARS,40,5858,buy_it_now,gold_special,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MLA618887655,MLA,Pc Armada Cpu Amd Full Gamer A10-9700 16gb Ddr...,53999.00,None,ARS,250,500,buy_it_now,gold_pro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MLA855055612,MLA,Notebook Exo Smart E19 Intel Celeron Ram4gb Ss...,39999.00,None,ARS,500,5000,buy_it_now,gold_pro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MLA841426834,MLA,Pasta Térmica Arctic Mx-4 4grs. Alto Rendimien...,1600.00,None,ARS,500,500,buy_it_now,gold_special,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10478,MLA678679714,MLA,Auriculares Genius Hs 05a Headset Control Volu...,2249.00,None,ARS,1,500,buy_it_now,gold_special,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10479,MLA858821035,MLA,The Lego Movie Videogame Formato Físico Ps4 Or...,2999.00,None,ARS,1,100,buy_it_now,gold_pro,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10480,MLA861128302,MLA,Call Of Duty Black Ops 2 Ps3 Gold | Español | ...,449.95,None,ARS,500,100,buy_it_now,gold_special,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10481,MLA818759510,MLA,Cable Cargador Y Transferencia Ps Vita Alto Re...,388.99,None,ARS,1,100,buy_it_now,gold_special,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Siendo que este proceso toma tiempo, lo mejor va a ser guardar el results_df en un CSV

In [8]:
results_df.to_csv("raw_dataset.csv", sep="|", header=True, index=True)